In [ ]:
#@title
import copy
import math
import random
from math import exp

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

z = []  
y = []  


def is_feasible(x):
    for j in range(m):
        if np.sum(x[:, j]) == 0:  
            return False

    for i in range(n):
        sum = 0
        for j in range(m):
            sum += x[i][j] * s[j]
        if sum > c[i]:            
            return False

    return True

def add_z(i, k, val, x):          
    # [m, n]
    x[i * n + k] += val


def add_y(i, j, k, val, x):       
    # [m, l, n]
    x[n * m + i * l * n + j * n + k] += val


def solve_yz(greedy=False):
    N = (l + 1) * n * m
    A_ub = []
    B_ub = []
    A_eq = []
    B_eq = []

    def eq_1():

        vs = []
        b = []
        for j in range(l):
            for k in range(n):
                v = [0] * N
                for i in range(m):
                    add_y(i, j, k, 1, v)
                vs += [v]
                b += [1]

        return vs, b

    def ineq_1():

        vs = []
        b = []
        for k in range(n):
            v = [0] * N
            for i in range(m):
                add_z(i, k, 1, v)
            vs += [v]
            b += [f[k] * max(x[k, :])]

        return vs, b
    
    def ineq_2():

        vs = []
        b = []
        for k in range(n):
            for i in range(m):
                v = [0] * N
                add_z(i, k, 1, v)
                vs += [v]
                b += [x[k][i] * f[k]]

        return vs, b

    def ineq_3():

        vs = []
        b = []
        for i in range(m):
            for k in range(n):
                for j in range(l):
                    v = [0] * N
                    add_y(i, j, k, -1, v)
                    b += [0]
                    vs += [v]

        return vs, b

    def ineq_4():

        vs = []
        b = []
        for i in range(m):
            for k in range(n):
                for j in range(l):
                    v = [0] * N
                    add_y(i, j, k, 1, v)
                    b += [x[k][i]]
                    vs += [v]

        return vs, b

    def ineq_5():

        vs = []
        b = []
        for i in range(m):
            for k in range(n):
                v = [0] * N
                for j in range(l):
                    add_y(i, j, k, la[i][j], v)
                add_z(i, k, -1.0 / r[i], v)
                vs += [v]
                b += [0]

        return vs, b

    def ineq_6():

        vs = []
        b = []
        for i in range(m):
            for k in range(n):
                if x[k][i] == 1:
                    for j in range(l):
                        v = [0] * N
                        for o in range(l):
                            add_y(i, o, k, la[i][o] * (d[i] - t[j][k]), v)
                        add_z(i, k, (t[j][k] - d[i]) / r[i], v)
                        vs += [v]
                        b += [-1]

        return vs, b

    def cost():
        v = [0] * N
        for i in range(m):
            for k in range(n):
                add_z(i, k, x[k][i] * Pf[k] / f[k], v)

        return v

    eq_greedy = ineq_1

    if greedy:
        eqs = [eq_1(), eq_greedy()]
        ineqs = [ineq_2(), ineq_3(), ineq_4(), ineq_5(), ineq_6()]
    else:
        eqs = [eq_1()]
        ineqs = [ineq_1(), ineq_2(), ineq_3(), ineq_4(), ineq_5(), ineq_6()]

    for ineq in ineqs:
        A_ub += ineq[0]
        B_ub += ineq[1]

    for eq in eqs:
        A_eq += eq[0]
        B_eq += eq[1]

    res = optimize.linprog(cost(), A_ub=A_ub, b_ub=B_ub, A_eq=A_eq, b_eq=B_eq)
    
    return res

def total_cost(x, g):
    z = np.reshape(g.x[:n * m], (m, n))

    cost = 0
    for k in range(n):
        for i in range(m):
            cost += x[k][i] * (Pc[k] * s[i] / c[k] + Pf[k] * z[i][k] / f[k])

    return cost


def guess_x_gibbs(epochs, omega=1e-6, greedy=False):
    global x

    cost_list = []
    best_g = None

    
    while True:
        x = np.random.randint(2, size=(n, m))
        if is_feasible(x):
          g = solve_yz(greedy)
          if g.success:
              best_g = g
              break


    print("[INIT]\t\t: cost=%s" % (total_cost(x, g)))

    for i in range(epochs):
        choice_idx = random.randint(0, n - 1)
        old_x = copy.deepcopy(x)
        x[choice_idx, :] = np.random.randint(2, size=m)
        new_g = solve_yz(greedy)

        if new_g.success and is_feasible(x):
            old_cost = total_cost(old_x, g)
            new_cost = total_cost(x, new_g)
            delta = new_cost - old_cost
            trans_prob = 1.0 / (1.0 + np.exp(1.0 * delta / omega))

            if random.random() <= trans_prob:
                print("[%3d][CHANGE]\t: new_cost=%s, old_cost=%s, prob=%s -- %s" % (i, new_cost, old_cost, trans_prob, new_cost))
                g = new_g
                best_g = g
            else:
                print("[%3d][KEEP]\t\t: new_cost=%s, old_cost=%s, prob=%s -- %s" % (i, new_cost, old_cost, trans_prob, old_cost))
                x = old_x
        else:
            print("[%3d][SKIP]\t\t: %s" % (i, new_g.success))
            x = old_x

        cost_list.append(total_cost(x, g))

    return cost_list 


def guess_x_random(nums, greedy=False):
    global x

    best_g = None 
    

    while True:
        x = np.random.randint(2, size=(n, m))
        if is_feasible(x):
          g = solve_yz(greedy)
          if g.success:
              best_g = g
              break
          else:
            print("no success g!")
        else:
          print("not feasible x!")

    print("INIT x = %s" % x)
    min_cost = total_cost(x, g)
    cost_list = []

    for i in range(nums):
        x = np.random.randint(2, size=(n, m))
        g = solve_yz(greedy)
        if g.success and is_feasible(x):
            cur_cost = total_cost(x, g)
            if cur_cost < min_cost:
                min_cost = cur_cost
                best_g = g

        cost_list.append(min_cost)
        print("[%3d]: %s --- %s" % (i, g.fun, min_cost))

    return cost_list 

def resource_usage_rate(g):
    z = np.reshape(g.x[:n * m], (m, n))
    
    usages = []
    for j in range(n): 
        sum = 0
        for i in range(m):
            sum += z[i][j]
        usages.append(sum / f[j])
    
    return usages


In [ ]:
from itertools import permutations  


def solve_greedy():
    
    services = list(range(m))
    services.sort(key=lambda x: d[x])
    
    server_order = [list(range(n)) for i in range(l)]
    for bs in range(l):
        server_order[bs].sort(key=lambda sv: t[bs][sv] * Pc[sv] / c[sv] * Pf[sv] / f[sv])
    
    store = c.tolist()
    compute = f.tolist()
    
    sum_cost = 0
    for svc in services:
        cost, store, compute = get_best_for_service(svc, store, compute, server_order)
        sum_cost += cost
    
    return sum_cost


def get_best_for_service(svc, store, compute, server_order):
    min_cost = float('inf')
    bst_store = None
    bst_compute = None
    
    for bs_list in permutations(range(l)):
        t_store = store.copy()
        t_compute = compute.copy()
        sum_cost = 0
        
        sv_set = set()
        for bs in bs_list:
            total_y = 1.0
            for sv in server_order[bs]:
                
                finished, cost = try_set_server(bs, svc, sv, t_store, t_compute, total_y, sv in sv_set)

                if finished > 0:
                    total_y -= finished
                    sum_cost += cost
                    sv_set.add(sv)
            
            assert total_y == 0, "impossible task ?"
        
        min_cost = min(min_cost, sum_cost)
        bst_store, bst_compute = t_store, t_compute
    
    return min_cost, bst_compute, bst_store


def try_set_server(bs, svc, sv, store, compute, total_y, free_store):
    if not free_store and s[svc] >= store[sv]:
        return 0, 0
    
    if compute[sv] == 0:
        return 0, 0
    
    st = s[svc] if not free_store else 0
    z = compute[sv]
    y = 1.0 / la[svc][bs] * (z / r[svc] - 1.0 / (d[svc] - t[bs][sv]))
    if y > total_y:
        z = r[svc] * (total_y * la[svc][bs] + 1.0 / (d[svc] - t[bs][sv]))
        y = total_y
    
    if y > 0:
        store[sv] -= st
        compute[sv] -= z
    
    return y, 1.0 * st / c[sv] * Pc[sv] + 1.0 * z / f[sv] * Pf[sv]
    
                
                
result_greedy = [solve_greedy()] * 5000